## Setup and Data Loading

In [ ]:
#imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

#file paths
file_path_non_prime = "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductNonPrime.csv"
file_path_prime = "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductPrime.csv"
file_path_walmart = "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductWalmart.csv"
file_path_tiktok = "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductTikTok.csv"

#data loading
data_non_prime = pd.read_csv(file_path_non_prime)
data_prime = pd.read_csv(file_path_prime)
data_walmart = pd.read_csv(file_path_walmart)
data_tiktok = pd.read_csv(file_path_tiktok)

#helper function that converts the date columns as numeric indices for modeling
def transform_dates(df):
    """Convert date columns to numeric indices for regression modeling."""
    return [int(col.replace('/', '')) for col in df.columns if col.startswith('10/') or col.startswith('11/') or col.startswith('12/')]


## Simplified Pricing Model

In [ ]:
#Simplified Pricing Model
def simplified_predict_prices(df, historical_dates=["10/7", "10/8", "10/9", "10/10", "10/11", "10/12", "10/13", "10/14"], target_dates=["11/26", "11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]):
    """
    Simplified pricing model: Uses mean prices of historical and surrounding days
    to predict prices for the target dates.
    """
    price_columns = historical_dates
    target_columns = target_dates
    results = df.copy()

    for index, row in df.iterrows():
        historical_prices = row[price_columns].dropna()
        if len(historical_prices) > 0:
            mean_price = historical_prices.mean()

            for target_day in target_columns:
                if target_day in results.columns:
                    results.loc[index, target_day] = mean_price
        else:
            for target_day in target_columns:
                if target_day in results.columns:
                    results.loc[index, target_day] = np.nan

    results[price_columns] = results[price_columns].interpolate(method="linear", limit_direction="both")
    return results

#run model
data_non_prime_simplified = simplified_predict_prices(data_non_prime)
data_prime_simplified = simplified_predict_prices(data_prime)
data_walmart_simplified = simplified_predict_prices(data_walmart)
data_tiktok_simplified = simplified_predict_prices(data_tiktok)

## Dynamic Pricing Model

In [ ]:
#Dynamic Pricing Model
def predict_dynamic_pricing(df, historical_dates=["10/7", "10/8", "10/9", "10/10", "10/11", "10/12", "10/13", "10/14"], target_dates=["11/26", "11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]):
    """Dynamic pricing based on historical trends using linear regression."""
    price_columns = historical_dates
    target_columns = target_dates
    date_indices = transform_dates(pd.DataFrame(columns=price_columns + target_columns))

    #data prep
    prices = df[price_columns].copy()
    prices.columns = date_indices[:len(price_columns)]
    imputer = SimpleImputer(strategy="mean")
    prices = pd.DataFrame(imputer.fit_transform(prices), columns=prices.columns)

    #fit the linear model and predict prices
    results = prices.copy()
    for i, row in prices.iterrows():
        historical_dates = row.dropna().index.to_list()
        historical_prices = row.dropna().values

        if len(historical_dates) > 1:
            model = LinearRegression()
            model.fit(np.array(historical_dates).reshape(-1, 1), historical_prices)
            target_date_indices = date_indices[len(price_columns):]
            predictions = model.predict(np.array(target_date_indices).reshape(-1, 1))
            results.loc[i, target_date_indices] = predictions

    results.columns = price_columns + target_columns
    return results.interpolate(method="linear", limit_direction="both")

#run model
data_non_prime_dynamic = predict_dynamic_pricing(data_non_prime)
data_prime_dynamic = predict_dynamic_pricing(data_prime)
data_walmart_dynamic = predict_dynamic_pricing(data_walmart)
data_tiktok_dynamic = predict_dynamic_pricing(data_tiktok)

## Value-Based Pricing Model

In [ ]:
#Value-Based Pricing Model
def predict_value_based_pricing(df, historical_dates=["10/7", "10/8", "10/9", "10/10", "10/11", "10/12", "10/13", "10/14"], target_dates=["11/26", "11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]):
    """Value-based pricing using polynomial regression."""
    price_columns = historical_dates
    target_columns = target_dates
    date_indices = transform_dates(pd.DataFrame(columns=price_columns + target_columns))

    #data prep
    prices = df[price_columns].copy()
    prices.columns = date_indices[:len(price_columns)]
    imputer = SimpleImputer(strategy="mean")
    prices = pd.DataFrame(imputer.fit_transform(prices), columns=prices.columns)

    #fit the polynomial regression and predict prices
    results = prices.copy()
    poly = PolynomialFeatures(degree=2)
    for i, row in prices.iterrows():
        historical_dates = row.dropna().index.to_list()
        historical_prices = row.dropna().values

        if len(historical_dates) > 1:
            poly_dates = poly.fit_transform(np.array(historical_dates).reshape(-1, 1))
            model = LinearRegression()
            model.fit(poly_dates, historical_prices)

            target_date_indices = date_indices[len(price_columns):]
            target_poly = poly.transform(np.array(target_date_indices).reshape(-1, 1))
            predictions = model.predict(target_poly)
            results.loc[i, target_date_indices] = predictions

    results.columns = price_columns + target_columns
    return results.interpolate(method="linear", limit_direction="both")

#run model
data_non_prime_value = predict_value_based_pricing(data_non_prime)
data_prime_value = predict_value_based_pricing(data_prime)

## Competition-Based Pricing Model

In [ ]:
#Competition-Based Pricing Model
def predict_competition_based_pricing(df, competitor_dfs, historical_dates=["10/7", "10/8", "10/9", "10/10", "10/11", "10/12", "10/13", "10/14"], target_dates=["11/26", "11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]):
    """Competition-based pricing using Random Forest regression."""
    price_columns = historical_dates
    target_columns = target_dates
    date_indices = transform_dates(pd.DataFrame(columns=price_columns + target_columns))

    #data prep
    prices = df[price_columns].copy()
    prices.columns = date_indices[:len(price_columns)]
    competitor_prices = pd.concat([comp[price_columns].mean() for comp in competitor_dfs], axis=1).mean(axis=1)
    competitor_prices.index = date_indices[:len(price_columns)]

    results = prices.copy()
    for i, row in prices.iterrows():
        historical_dates = row.dropna().index.to_list()
        historical_prices = row.dropna().values

        if len(historical_dates) > 1:
            model = RandomForestRegressor(n_estimators=100)
            model.fit(np.array(historical_dates).reshape(-1, 1), historical_prices)

            target_date_indices = date_indices[len(price_columns):]
            predictions = model.predict(np.array(target_date_indices).reshape(-1, 1))
            results.loc[i, target_date_indices] = predictions

    results.columns = price_columns + target_columns
    return results.interpolate(method="linear", limit_direction="both")

#run model
competitor_dfs = [data_walmart, data_tiktok]
data_prime_competition = predict_competition_based_pricing(data_prime, competitor_dfs)
data_non_prime_competition = predict_competition_based_pricing(data_non_prime, competitor_dfs)

## Results

In [7]:
#save all results to their respective files
output_paths = {
    "dynamic": {
        "non_prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductNonPrime_Dynamic.csv",
        "prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductPrime_Dynamic.csv",
        "walmart": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductWalmart_Dynamic.csv",
        "tiktok": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductTikTok_Dynamic.csv",
    },
    "value": {
        "non_prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductNonPrime_Value.csv",
        "prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductPrime_Value.csv",
    },
    "competition": {
        "non_prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductNonPrime_Competition.csv",
        "prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductPrime_Competition.csv",
    },
    "simplified": {
        "non_prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductNonPrime_Simplified.csv",
        "prime": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductPrime_Simplified.csv",
        "walmart": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductWalmart_Simplified.csv",
        "tiktok": "/home/jovyan/Capstone/Capstone/Data/MasterCapstoneData.xlsx - ProductTikTok_Simplified.csv",
    },
}

#save results for each model
data_non_prime_dynamic.to_csv(output_paths["dynamic"]["non_prime"], index=False)
data_prime_dynamic.to_csv(output_paths["dynamic"]["prime"], index=False)
data_walmart_dynamic.to_csv(output_paths["dynamic"]["walmart"], index=False)
data_tiktok_dynamic.to_csv(output_paths["dynamic"]["tiktok"], index=False)

data_non_prime_value.to_csv(output_paths["value"]["non_prime"], index=False)
data_prime_value.to_csv(output_paths["value"]["prime"], index=False)

data_non_prime_competition.to_csv(output_paths["competition"]["non_prime"], index=False)
data_prime_competition.to_csv(output_paths["competition"]["prime"], index=False)

data_non_prime_simplified.to_csv(output_paths["simplified"]["non_prime"], index=False)
data_prime_simplified.to_csv(output_paths["simplified"]["prime"], index=False)
data_walmart_simplified.to_csv(output_paths["simplified"]["walmart"], index=False)
data_tiktok_simplified.to_csv(output_paths["simplified"]["tiktok"], index=False)